In [1]:
BASE_URL = "http://192.168.4.1"

def encode_color(color_hex: str) -> str:
    return color_hex.replace("#", "%23")

In [16]:
# ESP32 ülekoormuse test
import requests
import time
import random

def random_color():
    return "#{:02X}{:02X}{:02X}".format(
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)
    )

def send_set_command():
    color = random_color()
    url = f"{BASE_URL}/set?value={encode_color(color)}"
    try:
        start = time.time()
        resp = requests.get(url, timeout=1)
        latency = time.time() - start
        return latency, resp.status_code
    except requests.exceptions.RequestException:
        return None, None

def test_overload():
    print("=== ESP32 ülekoormuse test ===")
    rates = [10, 20, 50, 75, 100, 150]  # käsk/sek
    stop = False

    for rate in rates:
        interval = 1.0 / rate
        print(f"\n📶 Saadame {rate} käske/sek")

        latencies = []
        timeouts = 0

        start_time = time.time()
        while time.time() - start_time < 5:
            latency, status = send_set_command()
            if latency is not None:
                latencies.append(latency)
            else:
                timeouts += 1
            time.sleep(interval)

        if not latencies:
            print("❌ ESP32 ei vasta üldse.")
            break

        avg_latency = sum(latencies) / len(latencies)
        max_latency = max(latencies)

        print(f"Keskmine viivitus: {avg_latency:.3f} sek")
        print(f"Max viivitus: {max_latency:.3f} sek")
        print(f"Kadu: {timeouts}")

        if avg_latency > 0.5 or timeouts > 0:
            print("❗ On tuvastatud ülekoormuse märgid.")
            stop = True
            break

    print("\n⏳ Taastumistest...")
    time.sleep(3)

    try:
        resp = requests.get(f"{BASE_URL}/get", timeout=2)
        if resp.status_code == 200:
            print("✅ Süsteem taastus.")
        else:
            print("❌ Süsteem pole taastunud.")
    except:
        print("❌ Ei saa vastust ESP32-st.")

test_overload()


=== ESP32 ülekoormuse test ===

📶 Saadame 10 käske/sek
Keskmine viivitus: 0.034 sek
Max viivitus: 0.036 sek
Kadu: 0

📶 Saadame 20 käske/sek
Keskmine viivitus: 0.033 sek
Max viivitus: 0.037 sek
Kadu: 0

📶 Saadame 50 käske/sek
Keskmine viivitus: 0.034 sek
Max viivitus: 0.051 sek
Kadu: 0

📶 Saadame 75 käske/sek
Keskmine viivitus: 0.035 sek
Max viivitus: 0.070 sek
Kadu: 0

📶 Saadame 100 käske/sek
Keskmine viivitus: 0.034 sek
Max viivitus: 0.037 sek
Kadu: 0

📶 Saadame 150 käske/sek
Keskmine viivitus: 0.033 sek
Max viivitus: 0.053 sek
Kadu: 0

⏳ Taastumistest...
✅ Süsteem taastus.


In [14]:
# LED värvi uuendamise visuaalse sujuvuse test
from IPython.display import display
import ipywidgets as widgets

FREQUENCIES_HZ = [1, 2, 5, 10, 20, 50]
DURATION = 5  
COLORS = ["#FF0000", "#00FF00", "#0000FF"]  # RGB

def cycle_colors(freq_hz):
    delay = 1.0 / freq_hz
    print(f"\nSageduse test: {freq_hz} Hz (viivitus {delay:.3f} sek)")

    start_time = time.time()
    i = 0
    while time.time() - start_time < DURATION:
        color = COLORS[i % len(COLORS)]
        try:
            requests.get(f"{BASE_URL}/set?value={encode_color(color)}", timeout=1)
        except:
            print("⚠️ Päringu viga")
        time.sleep(delay)
        i += 1

def run_visual_test():
    last_freq = 0
    
    print("=== LED värvi uuendamise visuaalse sujuvuse test ===")
    print("Jälgi LED värvi.\n")

    for freq in FREQUENCIES_HZ:
        cycle_colors(freq)
        answer = input("Kas värvimuutused olid sujuvad ja selgelt eristatavad? (jah/ei): ").strip().lower()
        if answer == "ei":
            print(f"\n❗ Optimaalne värvimuutmise kiirus veebiliidese jaoks: {last_freq} Hz")
            break
        else:
            last_freq = freq
    else:
        print("\n✅ Kõik sagedused kuni 50 Hz läbisid edukalt.")

# Запуск
run_visual_test()


=== LED värvi uuendamise visuaalse sujuvuse test ===
Jälgi LED värvi.


Sageduse test: 1 Hz (viivitus 1.000 sek)

Sageduse test: 2 Hz (viivitus 0.500 sek)

❗ Optimaalne värvimuutmise kiirus veebiliidese jaoks: 1 Hz
